In [ ]:
import time
import numpy as np
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

try:
    # Conectar ao servidor CoppeliaSim
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)

    # Handles do robô e rodas
    robotHandle = sim.getObject('Pioneer_p3dx')
    l_wheel = sim.getObject('Pioneer_p3dx_leftMotor')
    r_wheel = sim.getObject('Pioneer_p3dx_rightMotor')

    # Handles dos sensores ultra-sônicos
    sensorHandles = [sim.getObject(f'Pioneer_p3dx_ultrasonicSensor{i}') for i in range(1, 17)]

    # Constantes do robô
    L = 0.331
    r = 0.09751

    # Ganhos do controlador
    kv = 1.5
    kw = 3.5
    K_att = 1.0
    K_rep = 0.5

    # Posição do objetivo (em metros)
    x_goal = np.array([2.0, 2.0])  # altere para a posição desejada

    # Estado inicial
    q = np.array([0.0, 0.0, 0.0])  # x, y, theta

    # Iniciar simulação
    if sim.getSimulationState() != 0:
        sim.stopSimulation()
        time.sleep(1)
    sim.startSimulation()
    sim.step()

    print("Iniciando controle do robô...")
    while np.linalg.norm(x_goal - q[:2]) > 0.5:  # enquanto não estiver a 50 cm do goal
        sim_time = sim.getSimulationTime()
        dt = sim.getSimulationTimeStep()

        ### 1. FORÇA DE REPULSÃO ###
        F_rep = np.zeros(2)
        for sensor in sensorHandles:
            state, detectedPoint = sim.readProximitySensor(sensor)
            if state:  # obstáculo detectado
                detectedPoint = np.array(detectedPoint[:3])
                dist = np.linalg.norm(detectedPoint)
                if dist < 0.5:  # considerar apenas obstáculos próximos
                    direction = -detectedPoint[:2] / dist  # vetor para longe do obstáculo (2D)
                    F_rep += K_rep * direction * (1.0 / dist**2)  # força repulsiva proporcional

        ### 2. FORÇA DE ATRAÇÃO ###
        F_att = K_att * (x_goal - q[:2])

        ### 3. FORÇA RESULTANTE ###
        F = F_att + F_rep  # atração + repulsão (como na imagem 1)

        ### 4. CONTROLE CINEMÁTICO COM PSEUDO-INVERSA ###
        Fx, Fy = F
        theta = q[2]
        v = kv * (Fx * np.cos(theta) + Fy * np.sin(theta))
        w = kw * (np.arctan2(Fy, Fx) - theta)

        ### 5. CINEMÁTICA INVERSA PARA VELOCIDADES DAS RODAS ###
        wr = ((2.0 * v) + (w * L)) / (2.0 * r)
        wl = ((2.0 * v) - (w * L)) / (2.0 * r)

        # Enviar comandos
        sim.setJointTargetVelocity(l_wheel, wl)
        sim.setJointTargetVelocity(r_wheel, wr)

        ### 6. CINEMÁTICA DIRETA PARA ATUALIZAR POSIÇÃO ESTIMADA ###
        Mdir = np.array([
            [r * np.cos(q[2]) / 2, r * np.cos(q[2]) / 2],
            [r * np.sin(q[2]) / 2, r * np.sin(q[2]) / 2],
            [r / L, -r / L]
        ])
        u = np.array([wr, wl])
        q = q + (Mdir @ u) * dt

        print(f"[{sim_time:.2f}s] Pose: {q[:2]} θ={np.rad2deg(q[2]):.1f}°  | Dist Goal: {np.linalg.norm(x_goal - q[:2]):.2f} m")

        sim.step()

    ### FIM DA SIMULAÇÃO ###
    print("Parando robô...")
    sim.setJointTargetVelocity(l_wheel, 0)
    sim.setJointTargetVelocity(r_wheel, 0)

    print("Fim da simulação.")
    print('Posição estimada final:', q[:2], 'θ=', np.rad2deg(q[2]))

    sim.stopSimulation()

except Exception as e:
    print(f"Ocorreu um erro: {e}")

print("Programa finalizado.")
